In [61]:
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage
from typing import Annotated
from langchain_core.tools import InjectedToolArg
import json

In [25]:
load_dotenv()
# llm = ChatGroq(model="llama-3.1-8b-instant")
llm = ChatGroq(model="llama-3.1-8b-instant")

In [3]:
@tool
def multiply(a:int,b:int)->int:
    """Multiply two given numbers"""
    return a * b

@tool
def addition(a:int,b:int)->int:
    """Addition of two given numbers"""
    return a + b

In [4]:
def getTools():
    return [multiply,addition]

In [5]:
llm_with_tools = llm.bind_tools(getTools())

result = llm_with_tools.invoke("multiply 3 with 5")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tzcmjtyn2', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 279, 'total_tokens': 298, 'completion_time': 0.036560275, 'prompt_time': 0.018902738, 'queue_time': 0.052933012, 'total_time': 0.055463013}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3b0a28c4-1da8-4f38-87ce-1d8393ff9743-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'tzcmjtyn2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 19, 'total_tokens': 298})

In [6]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'tzcmjtyn2',
  'type': 'tool_call'}]

In [7]:
result.tool_calls[0]['args']

{'a': 3, 'b': 5}

In [8]:
multiply.invoke(result.tool_calls[0]['args'])

15

In [9]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='15', name='multiply', tool_call_id='tzcmjtyn2')

In [10]:
a = input("Enter a value")
print(a)

Enter a value 5


5


In [46]:
messages = []
val = 1;
while(val == 1):
    query = input("Prompt your query here")
    messages.append(HumanMessage(query))
    result = llm_with_tools.invoke(messages)
    messages.append(result)

    val = int(input("1-continue    0-stop"))

Prompt your query here mulitply 88 with 77
1-continue    0-stop 1
Prompt your query here multiply 43 with 22
1-continue    0-stop 0


In [39]:
messages

[HumanMessage(content='multiply 18 with 7', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'gx7g42nyv', 'function': {'arguments': '{"a":18,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 279, 'total_tokens': 298, 'completion_time': 0.026266382, 'prompt_time': 0.015447127, 'queue_time': 0.050780323, 'total_time': 0.041713509}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--35cb5e5f-a757-4aa6-87a2-3660770451c8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 18, 'b': 7}, 'id': 'gx7g42nyv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 19, 'total_tokens': 298}),
 HumanMessage(content='add 588 with 2283', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_cal

In [63]:
multiply.invoke(messages[1].tool_calls[0])

ToolMessage(content='6776', name='multiply', tool_call_id='3359m3zpn')

In [64]:
messages[1].tool_calls

[{'name': 'multiply',
  'args': {'a': 88, 'b': 77},
  'id': '3359m3zpn',
  'type': 'tool_call'}]

In [62]:
llm_with_tools.invoke(messages).content

'The result is 946.'

## Currency convertor

In [8]:
import requests
url = f'https://v6.exchangerate-api.com/v6/bb8909cf12d80e88629ff763/pair/USD/INR'

response = requests.get(url)

In [9]:
response.json()

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1763856001,
 'time_last_update_utc': 'Sun, 23 Nov 2025 00:00:01 +0000',
 'time_next_update_unix': 1763942401,
 'time_next_update_utc': 'Mon, 24 Nov 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.4539}

In [71]:
@tool
def get_conversion_rate(base_currency:str, target_currency:str) -> float:
    """This tool is going to give the conversion rate between base currency and target currency"""
    url = f'https://v6.exchangerate-api.com/v6/bb8909cf12d80e88629ff763/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency:int,conversion_rate:Annotated[float, InjectedToolArg]) -> float:
    """convert the base currency to the target currency"""
    return base_currency * conversion_rate

In [72]:
get_conversion_rate.invoke({"base_currency":"USD","target_currency":"INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1763856001,
 'time_last_update_utc': 'Sun, 23 Nov 2025 00:00:01 +0000',
 'time_next_update_unix': 1763942401,
 'time_next_update_utc': 'Mon, 24 Nov 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.4539}

In [73]:
convert.invoke({"base_currency":100,"conversion_rate":89.4539})

8945.390000000001

In [74]:
llm_with_tools = llm.bind_tools([get_conversion_rate,convert])

In [75]:
messges = [HumanMessage("can you convert 10 usd to inr based on the coversion rate")]
ai_msg = llm_with_tools.invoke(messges)

In [76]:
ai_msg.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': '23nqzybst',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10},
  'id': '8xsgextv9',
  'type': 'tool_call'}]

In [82]:
conversion_rate = 0
for msg in ai_msg.tool_calls:
    if msg['name'] == "get_conversion_rate":
        conversion_data = get_conversion_rate.invoke(msg)
        conversion_rate = json.loads(conversion_data.content)['conversion_rate']
    else:
        msg['args']['conversion_rate'] = float(conversion_rate)
        
        result = convert.invoke(msg)

In [85]:
result.content

'894.539'